## Определение стоимости автомобилей

### План

<a href='#section1'>1. Подготовка данных</a>

<a href='#section2'>2. Обучение моделей</a>

<a href='#section3'>3. Анализ моделей</a>

<a href='#section4'>4. Вывод</a>

<a id='section1'></a>
### 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import time
import warnings
import numpy as np
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/autos.csv')
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', 'NumberOfPictures'], axis = 1)
df = df.rename(columns = {'Price': 'price',
              'VehicleType': 'vehicle_type',
              'RegistrationYear': 'registration_year',
              'Gearbox': 'gearbox',
              'Power': 'power',
              'Model': 'model',
              'Kilometer': 'kilometer',
              'RegistrationMonth': 'registration_month',
              'FuelType': 'fuel_type',
              'Brand': 'brand',
              'NotRepaired': 'not_repaired'})
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
price                 354369 non-null int64
vehicle_type          316879 non-null object
registration_year     354369 non-null int64
gearbox               334536 non-null object
power                 354369 non-null int64
model                 334664 non-null object
kilometer             354369 non-null int64
registration_month    354369 non-null int64
fuel_type             321474 non-null object
brand                 354369 non-null object
not_repaired          283215 non-null object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


In [3]:
df.isnull().sum()

price                     0
vehicle_type          37490
registration_year         0
gearbox               19833
power                     0
model                 19705
kilometer                 0
registration_month        0
fuel_type             32895
brand                     0
not_repaired          71154
dtype: int64

In [4]:
p = pd.pivot_table(df, values='power', index=['model']).reset_index()
def power(elem):
    if (elem['power'] == 0) and (p[p['model'] == elem['model']].values.shape[0] == 1):
        return p[p['model'] == elem['model']]['power'].values[0] 
    return elem['power']


df['vehicle_type'] =  df['vehicle_type'].fillna('None')
df['gearbox'] =  df['gearbox'].fillna('None')
df['model'] =  df['model'].fillna('None')
df['fuel_type'] =  df['fuel_type'].fillna(df['fuel_type'].mode()[0])
df['not_repaired'] =  df['not_repaired'].fillna(df['not_repaired'].mode()[0])
df = df[df['price'] != 0]
df['power'] = df.apply(power, axis = 1)
df.head()


,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired
0,480,None,1993,manual,97.542898,golf,150000,0,petrol,volkswagen,no
1,18300,coupe,2011,manual,190.000000,None,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163.000000,grand,125000,8,gasoline,jeep,no
3,1500,small,2001,manual,75.000000,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69.000000,fabia,90000,7,gasoline,skoda,no


In [5]:
# Обработка категориальных признаков техникой One-Hot Encoding
data_ohe = pd.get_dummies(df, drop_first = True)

In [6]:
# Отделение целевого признака от остальных
target = data_ohe['price']
features = data_ohe.drop('price', axis=1)

In [7]:
# Разделение данных на выборки
features_temp, features_valid, target_temp, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345) 
features_train, features_test, target_train, target_test = train_test_split(
    features_temp, target_temp, test_size=0.25, random_state=12345) 
print('Размеры полученных наборов:')
print('тренировочный - ', features_train.shape[0])
print('валидационный - ', features_valid.shape[0])
print('тестовый - ', features_test.shape[0])

Размеры полученных наборов:
тренировочный -  206157
валидационный -  68720
тестовый -  68720


In [8]:
# Столбцы с числовыми данными
numeric = ['registration_year', 'power', 'kilometer', 'registration_month']

# Масштабирование данных
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_train.head()

,registration_year,power,kilometer,registration_month,vehicle_type_bus,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,...,brand_skoda,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,not_repaired_yes
14172,-0.113125,0.172043,0.576578,1.140729,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
184306,-0.038537,-0.024240,0.576578,0.870319,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
130200,0.011189,0.114313,0.576578,1.681548,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
77136,-0.038537,-0.110836,0.576578,0.059091,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91423,-0.026105,-0.019307,0.576578,0.870319,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Файл с данными содержит более 350 тысяч строк и 11 столбцов.

Пропуски в колонках с типом автомобильного кузова, коробки передач и моделью автомобиля заполним строкой None, чтобы не удалять эти строки. Тип топлива это чаще всего нефть - заполним модой. Если машина была в ремонте, наверное это указали бы, также заполним модой. Удалим строки таблицы с нулевой ценой. Пропуски в столбце мощности автомобиля заполним средним для данной марки.

Категориальных признаки преобразуем в числовые, а числовые масштабируем.

Отделяем целевой признак - это цена. 

Разделяем набор данных на обучающую, валидационную и тестовую выборки в соотношении 60:20:20.

<a id='section2'></a>
### 2. Обучение моделей

In [9]:
result = pd.DataFrame(columns=['Алгоритм',
                               'Качество предсказания',
                               'Скорость обучения, сек',
                               'Время предсказания, сек',
                               'Выборка'])

In [10]:
start_learn = time.time()
model = LinearRegression()
model.fit(features_train, target_train)
end_learn = round(time.time() - start_learn, 2)

print('Время обучения модели линейной регрессии: {}'.format(end_learn))

Время обучения модели линейной регрессии: 27.16


In [11]:
start_predict = time.time()
predictions = model.predict(features_valid)
end_predict = round(time.time() - start_predict, 2)
rmse = int(mean_squared_error(predictions, target_valid)**0.5)

result.loc[len(result)] = ('Линейная регрессия', rmse, end_learn, end_predict, 'валидационная') 
print('RMSE линейной регрессии на валидационной выборке - ', rmse)
print('Время предсказания линейной регрессии: {}'.format(end_predict))

RMSE линейной регрессии на валидационной выборке -  3164
Время предсказания линейной регрессии: 0.21


#### Подбор лучших гиперпараметров:

In [12]:
%%time
estimator = LGBMRegressor()
params_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40],
    'max_depth': [5, 10, 20]
}

gbm = GridSearchCV(estimator, params_grid, cv=3)
gbm.fit(features_train, target_train)
params = gbm.best_params_
print('Лучшие параметры для модели LGBMRegressor:', gbm.best_params_)

Лучшие параметры для модели LGBMRegressor: {'learning_rate': 1, 'max_depth': 10, 'n_estimators': 40}
CPU times: user 21min 59s, sys: 27.5 s, total: 22min 27s
Wall time: 22min 37s


In [13]:
df = df.astype({'vehicle_type':'category',
                'gearbox':'category',
                'model':'category',
                'fuel_type':'category',
                'brand':'category',
                'not_repaired':'category'}) 
# Отделение целевого признака от остальных
target = df['price']
features = df.drop('price', axis=1)
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features[numeric] = scaler.transform(features[numeric])
features.head()

,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired
0,None,1993.0,manual,97.542898,golf,150000.0,8.378708e-17,petrol,volkswagen,no
1,coupe,2011.0,manual,190.000000,None,125000.0,5.000000e+00,gasoline,audi,yes
2,suv,2004.0,auto,163.000000,grand,125000.0,8.000000e+00,gasoline,jeep,no
3,small,2001.0,manual,75.000000,golf,150000.0,6.000000e+00,petrol,volkswagen,no
4,small,2008.0,manual,69.000000,fabia,90000.0,7.000000e+00,gasoline,skoda,no


In [14]:
model = LGBMRegressor(learning_rate = params['learning_rate'],
                          n_estimators = params['n_estimators'],
                          max_depth = params['max_depth'])

final_score = np.mean(cross_val_score(model, features, target, cv = 3))
print('Средняя оценка качества модели:', final_score)


features_temp, features_valid, target_temp, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345) 
features_train, features_test, target_train, target_test = train_test_split(
    features_temp, target_temp, test_size=0.25, random_state=12345) 

Средняя оценка качества модели: 0.8393915879011509


In [20]:
start_learn = time.time()
model = LGBMRegressor(learning_rate = params['learning_rate'],
                          n_estimators = params['n_estimators'],
                          max_depth = params['max_depth'])
model.fit(features_train, target_train)
end_learn = round(time.time() - start_learn, 2)

print('Время обучения модели LGBMRegressor: {}'.format(end_learn))

Время обучения модели LGBMRegressor: 186.65


In [21]:
start_predict = time.time()
predictions = model.predict(features_valid)
end_predict = round(time.time() - start_predict, 2)
rmse = int(mean_squared_error(predictions, target_valid)**0.5)

result.loc[len(result)] = ('LGBMRegressor', rmse, end_learn, end_predict, 'валидационная') 
print('RMSE LGBMRegressor на валидационной выборке - ', rmse)
print('Время предсказания LGBMRegressor - {}'.format(end_predict))

RMSE LGBMRegressor на валидационной выборке -  1810
Время предсказания LGBMRegressor - 0.26


<div class="alert alert-block alert-warning">
Преобразовала признаки в категориальные и обучила модель с ними.
    
Применила метод кросс-валидации дополнительно, только метрика качества другого порядка получилась.    
</div>

#### Проверка лучшей модели на тестовой выборке:

In [22]:
start_predict = time.time()
predictions = model.predict(features_test)
end_predict = round(time.time() - start_predict, 2)
rmse = int(mean_squared_error(predictions, target_test)**0.5)

result.loc[len(result)] = ('LGBMRegressor', rmse, end_learn, end_predict, 'тестовая') 
print('RMSE LGBMRegressor на тестовой выборке - ', rmse)
print('Время предсказания LGBMRegressor - {}'.format(end_predict))


RMSE LGBMRegressor на тестовой выборке -  1838
Время предсказания LGBMRegressor - 0.36


#### Проверка лучшей модели на адекватность:

In [23]:
dummy = DummyRegressor(strategy="median")
dummy.fit(features_train, target_train)
dummy_predict = dummy.predict(features_test)
dummy_rmse = int(mean_squared_error(dummy_predict, target_test)**0.5)
result.loc[len(result)] = ('DummyRegressor', dummy_rmse, '-', '-', 'тестовая')
print('RMSE DummyRegressor на тестовой выборке', dummy_rmse)

RMSE DummyRegressor на тестовой выборке 4849


Построили модели градиентного бустинга и линейной регресии, составили таблицу с результатами, подобрали лучшие гиперпараметры для модели LGBMRegressor, проверили эту модель на тестовой выборке и на адекватность.

<a id='section3'></a>
### 3. Анализ моделей

In [24]:
result

,Алгоритм,Качество предсказания,"Скорость обучения, сек","Время предсказания, сек",Выборка
0,Линейная регрессия,3164,27.16,0.21,валидационная
1,LGBMRegressor,1810,81.83,0.4,валидационная
2,LGBMRegressor,1838,81.83,0.39,тестовая
3,DummyRegressor,4849,-,-,тестовая
4,LGBMRegressor,1810,186.65,0.26,валидационная
5,LGBMRegressor,1838,186.65,0.36,тестовая
6,DummyRegressor,4849,-,-,тестовая


На нашем наборе данных модель градиентного бустинга показала результаты метрики RMSE намного лучше линейной регрессии, а также скорость обучения в несколько раз выше. Время предсказания у линейной регрессии примерно в три раза выше, чем у LGBMRegressor. На тестовой выборке лучшая модель показала почти такие же результаты, как и на валидационной. 

<a id='section4'></a>
### Вывод

В процессе работы была выполнена предобработка данных - часть колонок удалена, заполнены пропуски в данных.

Провели масштабирование числовых данных с помощью StandardScaler. 

Категориальные признаки преобразовали в числовые для обучения линейной регрессии, а для градиентного бустинга преобразовали их в тип category.

Обучены модели линейной регрессии и градиентного бустинга. Из них мы выбрали наилучшую модель - LGBMRegressor, проверили её на тестовой выборке, а также на адекватность. Выбраны лучшие гиперпараметры с помощьюGridSearchCV.

Была составлена таблица с результатами.